### 🔹 Intégration PostgreSQL & Elasticsearch : Concepts Généraux
#### 📌 Pourquoi intégrer PostgreSQL avec Elasticsearch ?
PostgreSQL est une base relationnelle idéale pour stocker des données avec des relations complexes, tandis qu'Elasticsearch est un moteur de recherche performant conçu pour des requêtes rapides et full-text. L'intégration des deux permet :

- Stocker les données structurées dans PostgreSQL.
- Utiliser Elasticsearch pour les recherches avancées et le full-text search.
- Garder les deux bases synchronisées, soit en batch (Logstash, Python), soit en temps réel (trigger PostgreSQL, Debezium).

### 🛠️ 1. Synchronisation des données entre PostgreSQL et Elasticsearch
Il existe plusieurs approches :

- Logstash + JDBC (solution no-code/basique, pas toujours temps réel).
- Service Python (elasticsearch-py) (plus flexible, mais demande du dev).
- Trigger PostgreSQL + API Elasticsearch (temps réel, performant).
- Debezium (Change Data Capture) (événements en temps réel).

### 📌 Tutoriel 1 : Synchronisation simple avec Logstash (Batch Mode)
##### 🔹 Prérequis :
- PostgreSQL installé avec une base de test.
- Elasticsearch installé et en fonctionnement.
- Logstash installé.
#### 🔹 Étapes :
- 1. Créer une table de test dans PostgreSQL :

In [ ]:
CREATE TABLE produits (
    id SERIAL PRIMARY KEY,
    nom VARCHAR(255),
    description TEXT,
    prix DECIMAL(10,2)
);

INSERT INTO produits (nom, description, prix) VALUES 
('Ordinateur', 'Un PC performant', 1200.99),
('Clavier mécanique', 'Un clavier RGB pour gamers', 150.00);

- 2. Créer un fichier de configuration Logstash :

Crée un fichier postgresql_to_es.conf :

```yaml
input {
  jdbc {
    jdbc_connection_string => "jdbc:postgresql://localhost:5432/ma_base"
    jdbc_user => "mon_user"
    jdbc_password => "mon_mdp"
    jdbc_driver_library => "/path/to/postgresql.jar"
    jdbc_driver_class => "org.postgresql.Driver"
    statement => "SELECT * FROM produits"
  }
}

output {
  elasticsearch {
    hosts => ["http://localhost:9200"]
    index => "produits"
  }
}
```

- 3. Exécuter Logstash :

```sh
logstash -f postgresql_to_es.conf
```

- Les données sont maintenant envoyées à Elasticsearch.

- 4. Vérifier dans Elasticsearch :

```sh
curl -X GET "localhost:9200/produits/_search?pretty"
```

### 📌 Tutoriel 2 : Mise à jour en temps réel avec un Trigger PostgreSQL

Si tu veux que PostgreSQL envoie directement des données mises à jour vers Elasticsearch, utilise un trigger + une fonction PL/pgSQL.

- 1.Créer une fonction PostgreSQL qui envoie des mises à jour à Elasticsearch :

In [ ]:
CREATE OR REPLACE FUNCTION notify_elasticsearch()
RETURNS TRIGGER AS $$
BEGIN
    PERFORM pg_notify('es_update', row_to_json(NEW)::text);
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

- 2. Créer un trigger sur la table :

In [ ]:
CREATE TRIGGER produits_update_trigger
AFTER INSERT OR UPDATE ON produits
FOR EACH ROW
EXECUTE FUNCTION notify_elasticsearch();

- 3. Créer un script Python pour écouter ces événements et envoyer à Elasticsearch :

In [ ]:
import psycopg2
import psycopg2.extras
import json
from elasticsearch import Elasticsearch

# Connexion à PostgreSQL
conn = psycopg2.connect("dbname=ma_base user=mon_user password=mon_mdp host=localhost")
cur = conn.cursor()

# Connexion à Elasticsearch
es = Elasticsearch(["http://localhost:9200"])

# Fonction pour écouter PostgreSQL
def listen():
    cur.execute("LISTEN es_update;")
    print("En attente d'événements PostgreSQL...")

    while True:
        conn.poll()
        while conn.notifies:
            notify = conn.notifies.pop(0)
            data = json.loads(notify.payload)
            es.index(index="produits", id=data["id"], body=data)
            print(f"Données mises à jour dans Elasticsearch: {data}")

listen()

- 4. Lancer le script en arrière-plan :

```sh
python sync_postgres_es.py
```

- 5. Tester:

In [ ]:
INSERT INTO produits (nom, description, prix) VALUES ('Souris Gamer', 'Une souris ultra rapide', 89.99);

- Cela enverra automatiquement la mise à jour à Elasticsearch !